<a href="https://cognitiveclass.ai"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/Logos/organization_logo/organization_logo.png" width = 400> </a>

<h1 align=center><font size = 5>Peer Review Final Assignment</font></h1>

## Introduction


In this lab, you will build an image classifier using the VGG16 pre-trained model, and you will evaluate it and compare its performance to the model we built in the last module using the ResNet50 pre-trained model. Good luck!

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>    

1. <a href="#item41">Download Data 
2. <a href="#item42">Part 1</a>
3. <a href="#item43">Part 2</a>  
4. <a href="#item44">Part 3</a>  

</font>
    
</div>

<a id="item41"></a>

## Download Data

Use the <code>wget</code> command to download the data for this assignment from here: https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

Use the following cells to download the data.

In [ ]:
!wget  https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0321EN/data/concrete_data_week4.zip

In [ ]:
!unzip concrete_data_week4.zip

After you unzip the data, you fill find the data has already been divided into a train, validation, and test sets.

<a id="item42"></a>

## Part 1

In this part, you will design a classifier using the VGG16 pre-trained model. Just like the ResNet50 model, you can import the model <code>VGG16</code> from <code>keras.applications</code>.

You will essentially build your classifier as follows:
1. Import libraries, modules, and packages you will need. Make sure to import the *preprocess_input* function from <code>keras.applications.vgg16</code>.
2. Use a batch size of 100 images for both training and validation.
3. Construct an ImageDataGenerator for the training set and another one for the validation set. VGG16 was originally trained on 224 × 224 images, so make sure to address that when defining the ImageDataGenerator instances.
4. Create a sequential model using Keras. Add VGG16 model to it and dense layer.
5. Compile the mode using the adam optimizer and the categorical_crossentropy loss function.
6. Fit the model on the augmented data using the ImageDataGenerators.

Use the following cells to create your classifier.

In [1]:
from keras.preprocessing.image import ImageDataGenerator
import keras
from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

In [2]:
num_classes = 2
image_resize = 224
batch_size_training = 100
batch_size_validation = 100

In [3]:
#Construct an ImageDataGenerator
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
)

In [4]:
#Construct an ImageDataGenerator -> training set
train_generator = data_generator.flow_from_directory(
    'concrete_data_week4/train',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical')

Found 30001 images belonging to 2 classes.


In [5]:
#Construct an ImageDataGenerator -> validation set
validation_generator = data_generator.flow_from_directory(
    'concrete_data_week4/valid',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_validation,
    class_mode='categorical')

Found 9501 images belonging to 2 classes.


In [6]:
#Create a sequential model
model = Sequential()

In [7]:
#Add VGG16 model and dense layer
model.add(VGG16(
    include_top=False,
    pooling='avg',
    weights='imagenet',
    ))


In [8]:
model.add(Dense(num_classes, activation="softmax"))
model.layers[0].trainable = False
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 dense (Dense)               (None, 2)                 1026      
                                                                 
Total params: 14,715,714
Trainable params: 1,026
Non-trainable params: 14,714,688
_________________________________________________________________


In [9]:
#Compile the mode using the adam optimizer & the categorical_crossentropy loss function
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(validation_generator)
num_epochs = 2
#Fit the model on the augmented data using the ImageDataGenerators.
fit_history = model.fit(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=validation_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
)

Epoch 1/2
301/301 [==============================] - 6800s 23s/step - loss: 0.1106 - accuracy: 0.9589 - val_loss: 0.0263 - val_accuracy: 0.9957
Epoch 2/2
301/301 [==============================] - 7155s 24s/step - loss: 0.0199 - accuracy: 0.9954 - val_loss: 0.0154 - val_accuracy: 0.9965


In [12]:
model.save("classifier_vgg16_model.h5")

<a id="item43"></a>

## Part 2

In this part, you will evaluate your deep learning models on a test data. For this part, you will need to do the following:

1. Load your saved model that was built using the ResNet50 model. 
2. Construct an ImageDataGenerator for the test set. For this ImageDataGenerator instance, you only need to pass the directory of the test images, target size, and the **shuffle** parameter and set it to False.
3. Use the **evaluate_generator** method to evaluate your models on the test data, by passing the above ImageDataGenerator as an argument. You can learn more about **evaluate_generator** [here](https://keras.io/models/sequential/).
4. Print the performance of the classifier using the VGG16 pre-trained model.
5. Print the performance of the classifier using the ResNet pre-trained model.


Use the following cells to evaluate your models.

In [14]:
from keras.models import load_model
# Step1: Load the saved ResNet50 model
resnet50_model = load_model('classifier_resnet_model.h5')

In [15]:
# Check its architecture
resnet50_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2048)              23587712  
                                                                 
 dense (Dense)               (None, 2)                 4098      
                                                                 
Total params: 23,591,810
Trainable params: 4,098
Non-trainable params: 23,587,712
_________________________________________________________________


In [16]:
#Step2: Construct an ImageDataGenerator for the test set.
test_generator = data_generator.flow_from_directory(
    'concrete_data_week4/test',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical',
    shuffle=False)

Found 500 images belonging to 2 classes.


In [18]:
#Step3: Use the evaluate_generator method to evaluate your models on the test data
#Step4: Print the performance of the classifier using the VGG16 pre-trained model.

performance_vgg16 = model.evaluate_generator(test_generator)
print("Performance of the VGG16-trained model")
print("Loss: {}".format(round(performance_vgg16[0], 5)))
print("Accuracy: {}".format(round(performance_vgg16[1], 5)))


<ipython-input-18-6eddabf8f424>:4: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  performance_vgg16 = model.evaluate_generator(test_generator)


Performance of the VGG16-trained model
Loss: 0.01311
Accuracy: 0.996


In [19]:
#Step5: Print the performance of the classifier using the ResNet pre-trained model.
performance_resnet50 = resnet50_model.evaluate(test_generator)
print("Performance of the ResNet50-trained model")
print("Loss: {}".format(round(performance_resnet50[0], 5)))
print("Accuracy: {}".format(round(performance_resnet50[1], 5)))

5/5 [==============================] - 35s 7s/step - loss: 0.0022 - accuracy: 1.0000
Performance of the ResNet50-trained model
Loss: 0.00224
Accuracy: 1.0


   This part can be checked by this link: https://github.com/TyMick/coursera-ai-capstone/blob/master/DL0321EN-4-1-Comparing-Models-py-v1.0.ipynb

<a id="item44"></a>

## Part 3

In this model, you will predict whether the images in the test data are images of cracked concrete or not. You will do the following:

1. Use the **predict_generator** method to predict the class of the images in the test data, by passing the test data ImageDataGenerator instance defined in the previous part as an argument. You can learn more about the **predict_generator** method [here](https://keras.io/models/sequential/).
2. Report the class predictions of the first five images in the test set. You should print something list this:

<center>
    <ul style="list-style-type:none">
        <li>Positive</li>  
        <li>Negative</li> 
        <li>Positive</li>
        <li>Positive</li>
        <li>Negative</li>
    </ul>
</center>

Use the following cells to make your predictions.

In [22]:
def print_prediction(prediction):
    if prediction[0] > prediction[1]:
        print("Negative ({}% certainty)".format(round(prediction[0] * 100, 1)))
    elif prediction[1] > prediction[0]:
        print("Positive ({}% certainty)".format(round(prediction[1] * 100, 1)))
    else:
        print("Unsure (prediction split 50–50)")

In [24]:
predictions_vgg16 = model.predict(test_generator)
print("First five predictions for the VGG16-trained model:")
for i in range(5):
    print_prediction(predictions_vgg16[i])

First five predictions for the VGG16-trained model:
Negative (99.7% certainty)
Negative (98.4% certainty)
Negative (98.1% certainty)
Negative (99.4% certainty)
Negative (99.0% certainty)


In [25]:
predictions_vgg16

array([[9.97464538e-01, 2.53546820e-03],
       [9.84309435e-01, 1.56905204e-02],
       [9.81420696e-01, 1.85792800e-02],
       [9.94493246e-01, 5.50673530e-03],
       [9.89515066e-01, 1.04849152e-02],
       [9.86364961e-01, 1.36349872e-02],
       [9.99265254e-01, 7.34770321e-04],
       [9.80644882e-01, 1.93551388e-02],
       [9.96921718e-01, 3.07819690e-03],
       [9.99703228e-01, 2.96697312e-04],
       [9.96477783e-01, 3.52221821e-03],
       [9.90896881e-01, 9.10311937e-03],
       [9.94344711e-01, 5.65529987e-03],
       [9.99558151e-01, 4.41891287e-04],
       [9.83427286e-01, 1.65727120e-02],
       [9.98219311e-01, 1.78063777e-03],
       [9.90290582e-01, 9.70944948e-03],
       [9.94431794e-01, 5.56819513e-03],
       [9.91730392e-01, 8.26961920e-03],
       [9.88071322e-01, 1.19287176e-02],
       [9.88545358e-01, 1.14546586e-02],
       [9.95425165e-01, 4.57487861e-03],
       [9.99073625e-01, 9.26302222e-04],
       [9.98540521e-01, 1.45948783e-03],
       [9.821819

In [27]:
predictions_resnet50 = resnet50_model.predict(test_generator)
print("First five predictions for the ResNet50-trained model:")
for i in range(5):
    print_prediction(predictions_resnet50[i])

First five predictions for the ResNet50-trained model:
Negative (100.0% certainty)
Negative (99.8% certainty)
Negative (99.7% certainty)
Negative (99.6% certainty)
Negative (100.0% certainty)


### Thank you for completing this lab!

This notebook was created by Alex Aklson.

This notebook is part of a course on **Coursera** called *AI Capstone Project with Deep Learning*. If you accessed this notebook outside the course, you can take this course online by clicking [here](https://cocl.us/DL0321EN_Coursera_Week4_LAB1).

<hr>

Copyright &copy; 2020 [IBM Developer Skills Network](https://cognitiveclass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).